In [1]:
#Import libreries 

import pandas as pd 
import numpy as np

from pandasql import sqldf # Hour SQL librery


import warnings
warnings.filterwarnings('ignore')

In [2]:
#import dataset 

df = pd.read_csv("Dataset/day.csv")
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [3]:
df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

[Dataset description](https://www.kaggle.com/datasets/lakshmi25npathi/bike-sharing-dataset?select=day.csv)

In [4]:
# control NA values 

print("- NA dataset exploritory",(df.isna().sum()),"\n\n") 
print("-Dataset shape:", df.shape,"\n")

- NA dataset exploritory instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64 


-Dataset shape: (731, 16) 



In [5]:
#convert into dataframe 
df = pd.DataFrame(df, columns=df.columns)
df_source = df.copy()

## Feature Generation/Extraction 

#### Categoria di feature (A) 

sono semplici da calcolare perché ragionano riga per riga, senza andare a includere righe precedenti o successive.

#### - Case 1 : 

- Feature “rents_by_month”: a partire dalle colonne “yr”, “mnth”, “casual” e “registered”, calcolare un valore numerico intero che rappresenta 
il numero totale di prese in prestito di bici, per ogni coppia (mese, anno). Deve essere suddiviso per utenti casuali e utenti registrati

In [6]:
#Method 1 

df_tmp = df[['yr','mnth','casual','registered']]
df_tmp.head(3)

,yr,mnth,casual,registered
0,0,1,331,654
1,0,1,131,670
2,0,1,120,1229


In [7]:
#Method 2

t = df.copy()
t_tmp = sqldf("Select yr, mnth , casual , registered from t")
t_tmp.head(3)

,yr,mnth,casual,registered
0,0,1,331,654
1,0,1,131,670
2,0,1,120,1229


In [8]:
#Mehod 1 

registered_sum = df.groupby(['yr','mnth'])['registered'].sum().reset_index()
registered_sum.head(3)

,yr,mnth,registered
0,0,1,35116
1,0,2,41973
2,0,3,51219


In [9]:
# Method 2

t_registered_sum = sqldf("""
                          SELECT yr,mnth, SUM(registered) 
                          FROM t 
                          GROUP BY yr,mnth
                         """)
t_registered_sum.head(3)

,yr,mnth,SUM(registered)
0,0,1,35116
1,0,2,41973
2,0,3,51219


In [10]:
#Method 1 

casual_sum=df.groupby(['yr','mnth'])['casual'].sum().reset_index()
casual_sum.head(3)

,yr,mnth,casual
0,0,1,3073
1,0,2,6242
2,0,3,12826


In [11]:
# Method 2 

t_casual_sum = sqldf("""
                        SELECT yr,mnth, SUM(casual)
                        FROM t
                        GROUP BY  yr,mnth
                        order by  yr,mnth asc
                        
                    """)
t_casual_sum.head(3)

,yr,mnth,SUM(casual)
0,0,1,3073
1,0,2,6242
2,0,3,12826


In [12]:
casual_sum = pd.DataFrame(casual_sum, columns=['yr','mnth','casual'])
casual_sum['casual'].head()

0     3073
1     6242
2    12826
3    22346
4    31050
Name: casual, dtype: int64

In [13]:
#Method 1 

df_tmp=casual_sum.merge(registered_sum, indicator = True,how = 'outer')
del df_tmp['_merge']

df_tmp.rename(columns = {'casual':'rents_by_month_casual','registered':'rents_by_month_registered'}, inplace = True)
df_tmp.head(3)

,yr,mnth,rents_by_month_casual,rents_by_month_registered
0,0,1,3073,35116
1,0,2,6242,41973
2,0,3,12826,51219


In [14]:
#Method 2 

t_df_tmp = sqldf("""
                        SELECT  *
                        FROM t_registered_sum 
                            LEFT JOIN t_casual_sum 
                            ON t_registered_sum.mnth = t_casual_sum.mnth
                        WHERE t_registered_sum.yr = t_casual_sum.yr
                 """)

t_df_tmp.rename(columns = {'SUM(casual)':'rents_by_month_casual','SUM(registered)':'rents_by_month_registered'}, inplace = True)

t_df_tmp = t_df_tmp.loc[:,~t_df_tmp.apply(lambda x: x.duplicated(),axis=1).all()].copy() # drop duplicated colunms 
t_df_tmp.head(3)

,yr,mnth,rents_by_month_registered,rents_by_month_casual
0,0,1,35116,3073
1,0,2,41973,6242
2,0,3,51219,12826


In [15]:
#Method 1 

df=df.merge(df_tmp, indicator = True,how = 'outer')
del df['_merge']
df.head(3)

t = df.copy()

In [16]:
#Mtehod 1

df[['yr','mnth','casual','rents_by_month_casual','registered','rents_by_month_registered']].sort_values(by='mnth').head()

,yr,mnth,casual,rents_by_month_casual,registered,rents_by_month_registered
0,0,1,331,3073,654,35116
368,1,1,95,8969,2273,87775
369,1,1,140,8969,3132,87775
370,1,1,307,8969,3791,87775
371,1,1,1070,8969,3451,87775


In [17]:
#Method 2 

t = sqldf("""
            SELECT yr,mnth,casual,rents_by_month_casual,registered,rents_by_month_registered
            FROM t
           --Order by mnth asc

           """ )
t.sort_values(by='mnth').head()

,yr,mnth,casual,rents_by_month_casual,registered,rents_by_month_registered
0,0,1,331,3073,654,35116
368,1,1,95,8969,2273,87775
369,1,1,140,8969,3132,87775
370,1,1,307,8969,3791,87775
371,1,1,1070,8969,3451,87775


#### - Case 2 : 

Feature “len_weathersit”: a partire dalla colonna “weathersit”, creare una variabile categorica così creata:
 
- 1 se la colonna contiene la stringa “cloudy” (minuscolo o maiuscolo)
- 2 se la colonna contiene la stringa “rain” (minuscolo o maiuscolo)
- 3 altrimenti

In [18]:
#Method 1

df['weathersit'] = ['cloudy' if df['weathersit'][i]==1 else 'rain' if df['weathersit'][i]==2 else 3 for i in df.index ]
df['weathersit'].value_counts()

cloudy    463
rain      247
3          21
Name: weathersit, dtype: int64

In [19]:
#Method 2
t = df_source.copy() 

t['weathersit']= sqldf("""
            SELECT
            CASE
                WHEN weathersit == 1 THEN 'cloudy'
                WHEN weathersit == 2 THEN 'rain'
                ELSE 3
            END 
            FROM t;
            """)

t['weathersit'].value_counts()

cloudy    463
rain      247
3          21
Name: weathersit, dtype: int64

In [20]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,rents_by_month_casual,rents_by_month_registered
0,1,2011-01-01,1,0,1,0,6,0,rain,0.344167,0.363625,0.805833,0.160446,331,654,985,3073,35116
1,2,2011-01-02,1,0,1,0,0,0,rain,0.363478,0.353739,0.696087,0.248539,131,670,801,3073,35116
2,3,2011-01-03,1,0,1,0,1,1,cloudy,0.196364,0.189405,0.437273,0.248309,120,1229,1349,3073,35116
3,4,2011-01-04,1,0,1,0,2,1,cloudy,0.200000,0.212122,0.590435,0.160296,108,1454,1562,3073,35116
4,5,2011-01-05,1,0,1,0,3,1,cloudy,0.226957,0.229270,0.436957,0.186900,82,1518,1600,3073,35116


#### - Case 3 : 

* Feature “original_hum”: a partire dalla colonna “hum”, creare una variabile numerica che contiene il valore originale della colonna. Usa la formula presente per calcolare l’inverso

In [21]:
df['original_hum'] = [str(df['hum'][i])[::-1]  for i in df.index]

In [22]:
df[['original_hum','hum']].head()

,original_hum,hum
0,338508.0,0.805833
1,780696.0,0.696087
2,372734.0,0.437273
3,534095.0,0.590435
4,759634.0,0.436957


#### - Case 4 : 

* Feature “rents_by_weekday”: a partire dalle colonne “weekday”, “casual” e “registered”, calcolare per ogni giorno della settimana se ci sono più bici prese in prestito da utenti casuali o registrati. Se possibile, ripetere questa analisi dividendola per stagioni dell’anno (colonna “season”)

In [23]:
df[['weekday', 'casual', 'registered']].head()

,weekday,casual,registered
0,6,331,654
1,0,131,670
2,1,120,1229
3,2,108,1454
4,3,82,1518


In [24]:
#method 1 

casual_by_weekday_sum=df.groupby(['weekday'])['casual'].sum().reset_index()
casual_by_weekday_sum.head(3)

,weekday,casual
0,0,140521
1,1,70784
2,2,57843


In [25]:
#Method 2 

t_casual_by_weekday_sum = sqldf("""
                                SELECT weekday , SUM(casual) casual
                                FROM t 
                                Group by weekday
                                """)

t_casual_by_weekday_sum.head(3)

,weekday,casual
0,0,140521
1,1,70784
2,2,57843


In [26]:
#Method 1 

registered_by_weekday_sum=df.groupby(['weekday'])['registered'].sum().reset_index()
registered_by_weekday_sum.head(3)

,weekday,registered
0,0,303506
1,1,384719
2,2,411266


In [27]:
#Method 2 

t_registered_by_weekday_sum = sqldf("""
                                SELECT weekday , SUM(registered) registered
                                FROM t 
                                Group by weekday
                                """)

t_registered_by_weekday_sum.head(3)

,weekday,registered
0,0,303506
1,1,384719
2,2,411266


In [28]:
casual_by_weekday_sum = casual_by_weekday_sum.astype('int64')
casual_by_weekday_sum

,weekday,casual
0,0,140521
1,1,70784
2,2,57843
3,3,57319
4,4,61460
5,5,78238
6,6,153852


In [29]:
#df['rents_by_weekday_casual']  = [True if df['casual'][i] > casual_by_weekday_sum['casual'][i] else False for i in df.index ]

## Categoria di feature (B)

#### - Case 5 : 

Questa categoria di feature (B) invece seguono la stessa logica che abbiamo visto oggi, quindi provo a valorizzare queste feature usando i valori presenti in 1 o più riga/he precedente/i o successiva/e:

* Feature “rents_3_days_before”: a partire dalle colonne “dteday” e “cnt”, riportare nella riga i-esima il valore della colonna “cnt” di tre giorni prima. 

  Esempio: se considero la riga con instant #4, devo trovare il valore “985” nella riga #4

In [30]:
df['rents_3_days_before'] = df['cnt'].shift(3)

In [31]:
df['rents_3_days_before']=df['rents_3_days_before'].dropna().reset_index(drop=True)

In [55]:
df=df.dropna().reset_index(drop=True)
df['casual'][10:20].mean()

92.0

#### - Case 6 : 

* Feature “avg_casual_rents_last_10_days”: a partire dalle colonne “dteday”, “casual”, calcolare la media del numero di affitti di bici negli ultimi 10 giorni (riga i-esima compresa)

    Esempio: se considero la riga con instant #10, devo trovare il valore “1251,5” nella riga #10

In [33]:
df['avg_casual_rents_last_10_days'] = [df['casual'][i:10+i].mean() for i in df.index]
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,...,hum,windspeed,casual,registered,cnt,rents_by_month_casual,rents_by_month_registered,original_hum,rents_3_days_before,avg_casual_rents_last_10_days
0,1,2011-01-01,1,0,1,0,6,0,rain,0.344167,...,0.805833,0.160446,331,654,985,3073,35116,338508.0,985.0,117.1
1,2,2011-01-02,1,0,1,0,0,0,rain,0.363478,...,0.696087,0.248539,131,670,801,3073,35116,780696.0,801.0,88.3
2,3,2011-01-03,1,0,1,0,1,1,cloudy,0.196364,...,0.437273,0.248309,120,1229,1349,3073,35116,372734.0,1349.0,77.7
3,4,2011-01-04,1,0,1,0,2,1,cloudy,0.200000,...,0.590435,0.160296,108,1454,1562,3073,35116,534095.0,1562.0,69.5
4,5,2011-01-05,1,0,1,0,3,1,cloudy,0.226957,...,0.436957,0.186900,82,1518,1600,3073,35116,759634.0,1600.0,64.1


#### - Case 7 : 

 Feature “is_last_2_days_hum”: a partire dalle colonne “dteday” e “hum”, creare un indicatore booleano valorizzato così:
        
* True se la media di umidità nella stessa stagione è superiore al valore medio degli ultimi 2 giorni
* False altrimenti

Il risultato è da mettere nella riga t (oggi)

In [56]:
media_umidita_nella_stessa_stagione = df.groupby(['season'])['hum'].mean().reset_index()
media_umidita_nella_stessa_stagione.head()

,season,hum
0,1,0.579918
1,2,0.626948
2,3,0.626096
3,4,0.694133


In [57]:
valore_medio_ultimi_2_giorni = df['hum'].tail(2).mean()
valore_medio_ultimi_2_giorni

0.569375

In [36]:
df['is_last_2_days_hum'] = (media_umidita_nella_stessa_stagione['hum']>valore_medio_ultimi_2_giorni)
df['is_last_2_days_hum'].value_counts()

True     311
False    283
Name: is_last_2_days_hum, dtype: int64

In [37]:
#df['is_last_2_days_hum'] =[True  if(media_umidita_nella_stessa_stagione['hum'][i]>valore_medio_ultimi_2_giorni) else False for i in df.index]
#df['is_last_2_days_hum'].value_counts()

#### - Case 8 : 

Feature “is_temp_5_days_ago_greater_than_atemp_today”: a partire dalle colonne “dteday”, “atemp” e “temp”, creare un indicatore booleano così valorizzato:
    
* True se il valore di temp a t-5 (5 giorni fa) è maggiore del valore di atemp a t (oggi)
* False altrimenti

Il risultato è da mettere nella riga t (oggi)

In [38]:
df['is_temp_5_days_ago_greater_than_atemp_today']=df['temp'].shift(5)
df['is_temp_5_days_ago_greater_than_atemp_today']=df['is_temp_5_days_ago_greater_than_atemp_today'].dropna().reset_index(drop=True)

In [39]:
df['is_temp_5_days_ago_greater_than_atemp_today'] = [True if df['is_temp_5_days_ago_greater_than_atemp_today'][i]>df['temp'][i] else False for i in df.index ]
df['is_temp_5_days_ago_greater_than_atemp_today'].value_counts()

False    728
Name: is_temp_5_days_ago_greater_than_atemp_today, dtype: int64

#### - Case 9 : 

Feature “windspeed_avg_last_month”: a partire dalle colonne “dteday” e “windspeed”, creare una variabile numerica che riporta in ogni riga il valore medio di “windspeed” calcolato nel mese precedente

In [40]:
df['windspeed_avg_last_month']=df['windspeed'].shift(-1)

In [41]:
df['windspeed_avg_last_month'] = [df['windspeed_avg_last_month'].mean() for i in df.index ]

In [42]:
df['windspeed_avg_last_month']=df['windspeed_avg_last_month'].dropna().reset_index(drop=True)

In [43]:
df['windspeed_avg_last_month'].head()

0    0.190447
1    0.190447
2    0.190447
3    0.190447
4    0.190447
Name: windspeed_avg_last_month, dtype: float64

#### - Case 10 : 

Feature “is_weathersit_equal_last_2_days”: a partire dalle colonne “dteday” e “weathersit”, creare un indicatore booleano così valorizzato:
    
* True se “weathersit” ha lo stesso valore nella riga i-esima e (i-1)-esima
* False altrimenti

Il risultato è da riportare nella riga i-esima

 

In [44]:
df[['dteday' ,'weathersit']].head()

,dteday,weathersit
0,2011-01-01,rain
1,2011-01-02,rain
2,2011-01-03,cloudy
3,2011-01-04,cloudy
4,2011-01-05,cloudy


In [58]:
df['is_weathersit_equal_last_2_days'] = df['weathersit'].shift(1)
df['is_weathersit_equal_last_2_days']=df['is_weathersit_equal_last_2_days'].dropna().reset_index(drop=True)

In [59]:
df['is_weathersit_equal_last_2_days'] =[True if df['is_weathersit_equal_last_2_days'][i] ==df['weathersit'][i] else False for i in df.index ]

In [60]:
df['is_weathersit_equal_last_2_days'].value_counts()

True     593
False      1
Name: is_weathersit_equal_last_2_days, dtype: int64

In [48]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,...,cnt,rents_by_month_casual,rents_by_month_registered,original_hum,rents_3_days_before,avg_casual_rents_last_10_days,is_last_2_days_hum,is_temp_5_days_ago_greater_than_atemp_today,windspeed_avg_last_month,is_weathersit_equal_last_2_days
0,1,2011-01-01,1,0,1,0,6,0,rain,0.344167,...,985,3073,35116,338508.0,985.0,117.1,False,False,0.190447,True
1,2,2011-01-02,1,0,1,0,0,0,rain,0.363478,...,801,3073,35116,780696.0,801.0,88.3,False,False,0.190447,False
2,3,2011-01-03,1,0,1,0,1,1,cloudy,0.196364,...,1349,3073,35116,372734.0,1349.0,77.7,False,False,0.190447,True
3,4,2011-01-04,1,0,1,0,2,1,cloudy,0.200000,...,1562,3073,35116,534095.0,1562.0,69.5,False,False,0.190447,True
4,5,2011-01-05,1,0,1,0,3,1,cloudy,0.226957,...,1600,3073,35116,759634.0,1600.0,64.1,False,False,0.190447,True
